In [4]:
import torch

import torch.nn as nn

import torch.nn.functional as F 

import numpy as np

In [5]:
# Tiny corpus

corpus = [
    "deep learning is powerful",
    "learning is fun and powerful",
    "deep learning is fun"
]


In [6]:
# Tokenize

tokens = " ".join(corpus).split()

vocab = sorted(set(tokens))

word_to_idx = {word: i for i, word in enumerate(vocab)}

idx_to_word = {i: word for word, i in word_to_idx.items()}

In [7]:
# Convert to sequences

seq_len = 3

data = []

In [8]:
for sentence in corpus:

    words = sentence.split()

    for i in range(len(words) - seq_len):

        input_seq = words[i: i + seq_len]

        target_word = words[i+seq_len]

        input_idx = [word_to_idx[word] for word in input_seq]

        target_idx = word_to_idx[target_word]

        data.append((input_idx, target_idx))


input_seqs = [torch.tensor(x) for x , _ in data]

target_words = [torch.tensor(y) for y ,_ in data]